In [2]:
pip install pandas numpy scikit-learn imbalanced-learn


Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

df = pd.read_csv("creditcard.csv")

In [3]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [7]:
df["Class"].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["Class"])
y = df["Class"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [10]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy=0.5, random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

In [11]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_sm, y_train_sm)

RandomForestClassifier(random_state=42)

In [20]:
import pickle
with open("fraud_detection_model.pkl", "wb") as file:
    pickle.dump(model, file)


In [22]:
from sklearn.metrics import classification_report, accuracy_score

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9994733330992591
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.83      0.87      0.85        98

    accuracy                           1.00     56962
   macro avg       0.92      0.93      0.92     56962
weighted avg       1.00      1.00      1.00     56962



In [28]:
import numpy as np
import pickle


with open("fraud_detection_model.pkl", "rb") as model_file:
    model = pickle.load(model_file)

def test_transaction(input_data):
    """Predicts if a transaction is fraudulent or legitimate."""
    input_data = np.array(input_data).reshape(1, -1)  
    
    prediction = model.predict(input_data)[0]
    return "⚠️ Fraudulent Transaction" if prediction == 1 else "✅ Legitimate Transaction"


default_values = [
    0.0,  
    -1.5, -0.1, 2.0, 
    1.2, -0.3, 0.4, 0.2, 0.1, 0.3, 0.0, 
    -0.5, -0.6, -1.0, -0.3, 1.4, -0.4, 0.2, 0.02, 0.4, 0.2,  
    -0.02, 0.3, -0.1, 0.06, 0.12, -0.18, 0.13, -0.02, 
    50.0  
]


print("\n🔍 Enter transaction details (only 4 values required):")

time = float(input("Enter Time: "))
v1 = float(input("Enter Feature V1: "))
v2 = float(input("Enter Feature V2: "))
v3 = float(input("Enter Feature V3: "))
amount = float(input("Enter Amount: "))

default_values[0] = time
default_values[1] = v1
default_values[2] = v2
default_values[3] = v3
default_values[-1] = amount

result = test_transaction(default_values)
print("\n🔍 Prediction:", result)


🔍 Enter transaction details (only 4 values required):


Enter Time:  1000
Enter Feature V1:  -2.1
Enter Feature V2:  1.5
Enter Feature V3:  -1.7
Enter Amount:  120.0



🔍 Prediction: ✅ Legitimate Transaction


C:\Users\mariu\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
